In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
torch.manual_seed(42)

df = pd.read_csv('/home/csgrad/kaushik3/LLM/Kaggle_63LLMs/OPT/human_opt.csv')

In [5]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
df['source'].value_counts()

Human       278018
OPT-1.3B     14783
OPT-30B      14539
OPT-2.7B      7292
OPT-6.7B      7143
OPT-125M      7048
OPT-350M      7033
OPT-13B       6410
Name: source, dtype: int64

In [7]:
AI = df[df['source'].str.startswith('OPT')]
AI.head()


,text,source,prompt_id,text_length,word_count,label_map
2,Huge amounts of digital videos are being produ...,OPT-30B,0,719,109,1
12,The smell of burned hair acummulated in Gordon...,OPT-30B,0,797,139,1
13,"If it is too warm, the flavors will not taste ...",OPT-125M,0,1181,211,1
21,Diane never wins anything. She was feeling luc...,OPT-13B,0,6312,834,1
27,Robertson (The Canadian): High levels of torqu...,OPT-30B,0,329,58,1


In [8]:
Human = df[df['source'] == 'Human']
print(len(Human))
Human = Human[:len(AI)]

278018


In [9]:
print(len(Human))
print(len(AI))

64248
64248


In [10]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [11]:
Human['embeddings'] = Human['text'].apply(generate_bert_embeddings)
Human['embeddings1'] = Human['embeddings'].apply(remove_outliers)
Human['normalized_embeddings'] = Human['embeddings1'].apply(min_max_normalize)
Human['reshaped_embeddings'] = Human['normalized_embeddings'].apply(reshape_embeddings)

In [8]:
AI['embeddings'] = AI['text'].apply(generate_bert_embeddings)
AI['embeddings1'] = AI['embeddings'].apply(remove_outliers)
AI['normalized_embeddings'] = AI['embeddings1'].apply(min_max_normalize)
AI['reshaped_embeddings'] = AI['normalized_embeddings'].apply(reshape_embeddings)

/tmp/ipykernel_1888842/1706120243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI['embeddings'] = AI['text'].apply(generate_bert_embeddings)
/tmp/ipykernel_1888842/1706120243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI['embeddings1'] = AI['embeddings'].apply(remove_outliers)
/tmp/ipykernel_1888842/1706120243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [9]:
AI_embeddings = AI['embeddings']

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/opt_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, AI_embeddings)
    

In [10]:
import matplotlib.pyplot as plt

for i in range(len(AI['reshaped_embeddings'])):
    plt.imshow(AI['reshaped_embeddings'].iloc[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/OPT/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


: 

In [3]:
import numpy as np

AI_embeddings = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/opt_embeddings.npy', allow_pickle=True)


In [6]:
for i in range(len(AI_embeddings)):
    
    AI_embeddings[i] = remove_outliers(AI_embeddings[i])
    AI_embeddings[i] = min_max_normalize(AI_embeddings[i])
    AI_embeddings[i] = reshape_embeddings(AI_embeddings[i])


In [7]:
import matplotlib.pyplot as plt

for i in range(49192, len(AI_embeddings)):
    plt.imshow(AI_embeddings[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/OPT/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


In [8]:
import os

folder_path = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/OPT'  # Replace with the actual folder path

file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print("Number of files in the folder:", file_count)


Number of files in the folder: 64248
